#Conexão Drive

In [1]:
from google.colab import drive

drive.mount("/content/gdrive")

Mounted at /content/gdrive


#Importação de Bibliotecas e Dados

In [2]:
#Importando módulos
import pandas as pd


In [3]:
#Carregando o DataSet
df = pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/pima-data.csv")

In [4]:
df.head(10)

,num_preg,glucose_conc,diastolic_bp,thickness,insulin,bmi,diab_pred,age,skin,diabetes
0,6,148,72,35,0,33.6,0.627,50,1.3790,True
1,1,85,66,29,0,26.6,0.351,31,1.1426,False
2,8,183,64,0,0,23.3,0.672,32,0.0000,True
3,1,89,66,23,94,28.1,0.167,21,0.9062,False
4,0,137,40,35,168,43.1,2.288,33,1.3790,True
5,5,116,74,0,0,25.6,0.201,30,0.0000,False
6,3,78,50,32,88,31.0,0.248,26,1.2608,True
7,10,115,0,0,0,35.3,0.134,29,0.0000,False
8,2,197,70,45,543,30.5,0.158,53,1.7730,True
9,8,125,96,0,0,0.0,0.232,54,0.0000,True


In [5]:
df.shape

(768, 10)

#Mapeamento das variáveis

In [6]:
#Mapeamento de variáveis
diabetes_map = {True : 1, False : 0}

In [7]:
#Aplicar o mapeamento
df['diabetes'] = df['diabetes'].map(diabetes_map)

In [8]:
df.head()

,num_preg,glucose_conc,diastolic_bp,thickness,insulin,bmi,diab_pred,age,skin,diabetes
0,6,148,72,35,0,33.6,0.627,50,1.3790,1
1,1,85,66,29,0,26.6,0.351,31,1.1426,0
2,8,183,64,0,0,23.3,0.672,32,0.0000,1
3,1,89,66,23,94,28.1,0.167,21,0.9062,0
4,0,137,40,35,168,43.1,2.288,33,1.3790,1


In [9]:
#Conferir como os dados estão distribuídos
num_true = len(df.loc[df['diabetes'] == True])
num_false = len(df.loc[df['diabetes'] == False])

print("Número de casos verdadeiros: {0} ({1:2.2f}%) ".format(num_true, (num_true/ (num_true + num_false)) * 100))
print("Número de casos falsos: {0} ({1:2.2f}%) ".format(num_false, (num_false/ (num_true + num_false)) * 100))

Número de casos verdadeiros: 268 (34.90%) 
Número de casos falsos: 500 (65.10%) 


#Spliting
###70% para treino e 30% teste

In [10]:
#scikit-learn 
from sklearn.model_selection import train_test_split

In [11]:
#Selecionar variaveis que preditoras () X
atributos = ['num_preg', 'glucose_conc', 'diastolic_bp', 'thickness',  'insulin', 'bmi', 'diab_pred', 'age']

In [12]:
# variável prevista (saída Y)
atrib_prev = ['diabetes']

In [13]:
#Criar objetos X e Y
X = df[atributos].values
Y = df[atrib_prev].values

In [14]:
X

array([[  6.   , 148.   ,  72.   , ...,  33.6  ,   0.627,  50.   ],
       [  1.   ,  85.   ,  66.   , ...,  26.6  ,   0.351,  31.   ],
       [  8.   , 183.   ,  64.   , ...,  23.3  ,   0.672,  32.   ],
       ...,
       [  5.   , 121.   ,  72.   , ...,  26.2  ,   0.245,  30.   ],
       [  1.   , 126.   ,  60.   , ...,  30.1  ,   0.349,  47.   ],
       [  1.   ,  93.   ,  70.   , ...,  30.4  ,   0.315,  23.   ]])

In [15]:
#Definir a taxa do Split
split_test_size = 0.30

In [16]:
#Criar os dados de treino e teste
X_treino, X_teste, Y_treino, Y_teste = train_test_split(X, Y, test_size = split_test_size, random_state = 42)

In [17]:
print("{0:0.2f}% nos dados de treino".format((len(X_treino)/len(df.index))*100))
print("{0:0.2f}% nos dados de teste".format((len(X_teste)/len(df.index))*100))

69.92% nos dados de treino
30.08% nos dados de teste


Valores Nulos

In [18]:
df.isnull().values.any()

False

In [19]:
#Falsos Nulos (Zero)
print(len(df.loc[df['glucose_conc'] == 0]))
print(len(df.loc[df['diastolic_bp'] == 0]))
print(len(df.loc[df['thickness'] == 0]))
print(len(df.loc[df['insulin'] == 0]))
print(len(df.loc[df['bmi'] == 0]))
print(len(df.loc[df['age'] == 0]))

5
35
227
374
11
0


In [20]:
#Tratando os dados Missing
from sklearn.impute import SimpleImputer

#Imputer: substitui os valores zero por algum dados relevante com base em alguma estratégia (mean, )

In [21]:
#criar o objeto e setar a estratégia
preenche_0 = SimpleImputer(missing_values=0, strategy='mean')

#Aplicar a estratégia
X_treino = preenche_0.fit_transform(X_treino)
X_teste = preenche_0.fit_transform(X_teste)

#Construindo o modelo 1 e treinando

In [22]:
from sklearn.naive_bayes import GaussianNB

In [23]:
#Criar uma instância do algoritmo de aprendizagem GaussianNB
modelo_v1 = GaussianNB()

In [24]:
#Treinando o MODELO - FIT
modelo_v1.fit(X_treino, Y_treino.ravel())

GaussianNB(priors=None, var_smoothing=1e-09)

Verificar exatidão do treino (acurácia)

In [25]:
# Pacote de métricas
from sklearn import metrics

In [26]:
#Acurácia do Modelo DO TREINO
nb_predict_train = modelo_v1.predict(X_treino)

In [27]:
print("Exatidão (Acurácia): {0:.4f}".format(metrics.accuracy_score(Y_treino, nb_predict_train)))

Exatidão (Acurácia): 0.7542


In [28]:
#Acurácia do Modelo de TESTE
nd_predict_test = modelo_v1.predict(X_teste)

In [29]:
print("Exatidão (Acurácia): {0:.4f}".format(metrics.accuracy_score(Y_teste, nd_predict_test)))

Exatidão (Acurácia): 0.7359


#Contruindo o modelo 2 

In [30]:
from sklearn.ensemble import RandomForestClassifier

In [31]:
modelo_v2 = RandomForestClassifier(random_state=42)
modelo_v2.fit(X_treino, Y_treino.ravel())

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [32]:
#Acurácia do Modelo
rf_predict_train = modelo_v2.predict(X_treino)

print("Exatidão (Acurácia): {0:.4f}".format(metrics.accuracy_score(Y_treino, rf_predict_train)))

Exatidão (Acurácia): 1.0000


In [33]:
rf_predict_test = modelo_v2.predict(X_teste)

print("Exatidão (Acurácia): {0:.4f}".format(metrics.accuracy_score(Y_teste, rf_predict_test)))

Exatidão (Acurácia): 0.7403


#Construindo o Modelo 3

In [34]:
#Usando ComplementNB (CNB)

from sklearn.naive_bayes import  ComplementNB

modelo_v3 = ComplementNB()

modelo_v3.fit(X_treino, Y_treino.ravel())

cnb_predict_train = modelo_v3.predict(X_treino)
cnb_predict_test = modelo_v3.predict(X_teste)

print("Exatidão (acuracia) Treino:  {0:.4f})".format(metrics.accuracy_score(Y_treino, cnb_predict_train)))
print("Exatidão (acuracia)Teste :  {0:.4f})".format(metrics.accuracy_score(Y_teste, cnb_predict_test)))



Exatidão (acuracia) Treino:  0.6443)
Exatidão (acuracia)Teste :  0.6234)


In [35]:
# Usando LARS Lasso

from sklearn import linear_model

modelo_v4 = linear_model.LassoLars()

modelo_v4.fit(X_treino, Y_treino.ravel())

lm_predict_train = modelo_v3.predict(X_treino)
lm_predict_test = modelo_v3.predict(X_teste)

print("Exatidão (acuracia): Treino  {0:.4f})".format(metrics.accuracy_score(Y_treino, lm_predict_train)))
print("Exatidão (acuracia): Teste {0:.4f})".format(metrics.accuracy_score(Y_teste, lm_predict_test)))

Exatidão (acuracia): Treino  0.6443)
Exatidão (acuracia): Teste 0.6234)


In [36]:
# Usando SGDClassifier

from sklearn.linear_model import SGDClassifier

modelo_v5 = linear_model.SGDClassifier()

modelo_v5.fit(X_treino, Y_treino.ravel())



sgdc_predict_train = modelo_v5.predict(X_treino)
sgdc_predict_test = modelo_v5.predict(X_teste)



print("Exatidão (acuracia): Treino  {0:.4f})".format(metrics.accuracy_score(Y_treino, sgdc_predict_train)))
print("Exatidão (acuracia): Teste  {0:.4f})".format(metrics.accuracy_score(Y_teste, sgdc_predict_test)))

Exatidão (acuracia): Treino  0.6480)
Exatidão (acuracia): Teste  0.6537)


#Conclusão

Responda aqui: qual o algoritmo que teve um melhor resultado, acurária?

De todos os usados o que se saiu melhor foi o 2modelo RandomForestClassifier